In [8]:
import pandas as pd
import ast
import numpy as np

In [9]:
movie_metadata = pd.read_csv('./data/movie.metadata.v1.csv')

movie_metadata.head()

,Freebase movie ID,Movie name,Movie release date,Year,Month,Day,Movie box office revenue,Movie runtime,Languages,Countries,Genres Reduced
0,/m/03vyhn,Ghosts of Mars,2001-08-24,2001.0,8.0,24.0,14010832.0,98.0,['English Language'],['United States of America'],"['Action', 'Thriller', 'Western', 'Adventure',..."
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,2000.0,2.0,16.0,NaN,95.0,['English Language'],['United States of America'],"['Thriller', 'Biography', 'Drama', 'Crime']"
2,/m/0crgdbh,Brun bitter,1988-01-01,1988.0,1.0,1.0,NaN,83.0,['Norwegian Language'],['Norway'],"['Crime', 'Drama', 'Fiction']"
3,/m/0285_cd,White Of The Eye,1987-01-01,1987.0,1.0,1.0,NaN,110.0,['English Language'],['United Kingdom'],"['Thriller', 'Psychological', 'Adult']"
4,/m/01mrr1,A Woman in Flames,1983-01-01,1983.0,1.0,1.0,NaN,106.0,['German Language'],['Germany'],['Drama']


In [10]:
# Function to safely convert string representations of lists into actual lists
def string_to_list(list_string):
    try:
        # This safely evaluates a string as a list
        return ast.literal_eval(list_string)
    except ValueError:
        # In case of error (e.g., empty strings), return an empty list
        return []
    
movie_metadata['Genres Reduced'] = movie_metadata['Genres Reduced'].apply(string_to_list)
movie_metadata['Languages'] = movie_metadata['Languages'].apply(string_to_list)
movie_metadata['Countries'] = movie_metadata['Countries'].apply(string_to_list)

movie_metadata.head()

,Freebase movie ID,Movie name,Movie release date,Year,Month,Day,Movie box office revenue,Movie runtime,Languages,Countries,Genres Reduced
0,/m/03vyhn,Ghosts of Mars,2001-08-24,2001.0,8.0,24.0,14010832.0,98.0,[English Language],[United States of America],"[Action, Thriller, Western, Adventure, Horror,..."
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,2000.0,2.0,16.0,NaN,95.0,[English Language],[United States of America],"[Thriller, Biography, Drama, Crime]"
2,/m/0crgdbh,Brun bitter,1988-01-01,1988.0,1.0,1.0,NaN,83.0,[Norwegian Language],[Norway],"[Crime, Drama, Fiction]"
3,/m/0285_cd,White Of The Eye,1987-01-01,1987.0,1.0,1.0,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Psychological, Adult]"
4,/m/01mrr1,A Woman in Flames,1983-01-01,1983.0,1.0,1.0,NaN,106.0,[German Language],[Germany],[Drama]


In [11]:
# Count the number of movies in each genre
genre_counts = movie_metadata['Genres Reduced'].apply(pd.Series).stack().value_counts()
genre_counts

C:\Users\arist\AppData\Local\Temp\ipykernel_47308\527503075.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  genre_counts = movie_metadata['Genres Reduced'].apply(pd.Series).stack().value_counts()


Drama            35570
Comedy           21915
Romance          11299
Thriller         10519
Action           10273
                 ...  
Reboot               9
Sponsored            8
Latino               6
Start Vehicle        2
Propaganda           2
Length: 78, dtype: int64

In [12]:
# # Setting a threshold for low sample size
# low_sample_threshold = 100

# # Identifying genres with low sample sizes
# low_sample_genres = genre_counts[genre_counts < low_sample_threshold]
# low_sample_genres_list = low_sample_genres.index.tolist()

# # Displaying genres considered to have low sample size
# low_sample_genres_list, low_sample_genres


In [14]:
# Setting a threshold for low sample size genres
low_sample_threshold = 100

# Identifying genres with low sample sizes
low_sample_genres = genre_counts[genre_counts < low_sample_threshold].index.tolist()

# Creating a bootstrapped dataframe for all genres below the threshold
# We will add extra values to each genre by resampling
additional_samples_per_genre = 500  # Additional samples for each genre

# Initialize an empty DataFrame to hold the bootstrapped data
bootstrapped_data_all_genres = pd.DataFrame()

# Performing bootstrap sampling for each low sample genre
for genre in low_sample_genres:
    genre_movies = movie_metadata[movie_metadata['Genres Reduced'].apply(lambda x: genre in x)]

    for _ in range(additional_samples_per_genre):
        sample = genre_movies.sample(n=1, replace=True)
        bootstrapped_data_all_genres = bootstrapped_data_all_genres.append(sample)

# Concatenating the original data with the bootstrapped data
enriched_data_all_genres = pd.concat([movie_metadata, bootstrapped_data_all_genres])

# Displaying the shape of the new, enriched dataframe
enriched_data_all_genres.shape


C:\Users\arist\AppData\Local\Temp\ipykernel_47308\2488593477.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bootstrapped_data_all_genres = bootstrapped_data_all_genres.append(sample)
C:\Users\arist\AppData\Local\Temp\ipykernel_47308\2488593477.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bootstrapped_data_all_genres = bootstrapped_data_all_genres.append(sample)
C:\Users\arist\AppData\Local\Temp\ipykernel_47308\2488593477.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bootstrapped_data_all_genres = bootstrapped_data_all_genres.append(sample)
C:\Users\arist\AppData\Local\Temp\ipykernel_47308\2488593477.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

(88741, 11)

In [27]:
genre = 'Propaganda'

# Display movies with the genre of interest
# genre_movies = enriched_data_all_genres[ genre in enriched_data_all_genres['Genres Reduced'] ]
# The correct way to do this is to use the apply method
genre_movies = enriched_data_all_genres[enriched_data_all_genres['Genres Reduced'].apply(lambda x: genre in x)]
genre_movies.head(100)

,Freebase movie ID,Movie name,Movie release date,Year,Month,Day,Movie box office revenue,Movie runtime,Languages,Countries,Genres Reduced
37264,/m/06w47fn,Winged Victory,1944-12-22,1944.0,12.0,22.0,NaN,130.0,[English Language],[United States of America],"[War, Politics, Propaganda, USA, Drama]"
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."
37264,/m/06w47fn,Winged Victory,1944-12-22,1944.0,12.0,22.0,NaN,130.0,[English Language],[United States of America],"[War, Politics, Propaganda, USA, Drama]"
37264,/m/06w47fn,Winged Victory,1944-12-22,1944.0,12.0,22.0,NaN,130.0,[English Language],[United States of America],"[War, Politics, Propaganda, USA, Drama]"
...,...,...,...,...,...,...,...,...,...,...,...
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."
37264,/m/06w47fn,Winged Victory,1944-12-22,1944.0,12.0,22.0,NaN,130.0,[English Language],[United States of America],"[War, Politics, Propaganda, USA, Drama]"
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."


In [28]:
genre_movies_before = movie_metadata[movie_metadata['Genres Reduced'].apply(lambda x: genre in x)]
genre_movies_before.head(100)

,Freebase movie ID,Movie name,Movie release date,Year,Month,Day,Movie box office revenue,Movie runtime,Languages,Countries,Genres Reduced
37264,/m/06w47fn,Winged Victory,1944-12-22,1944.0,12.0,22.0,NaN,130.0,[English Language],[United States of America],"[War, Politics, Propaganda, USA, Drama]"
65815,/m/04gnkhj,Meet the People,1944-06-01,1944.0,6.0,1.0,NaN,100.0,[English Language],[United States of America],"[Romance, Comedy, Propaganda, Black and White,..."


In [15]:
genre_counts_enriched = enriched_data_all_genres['Genres Reduced'].apply(pd.Series).stack().value_counts()
genre_counts_enriched

C:\Users\arist\AppData\Local\Temp\ipykernel_47308\457597570.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  genre_counts_enriched = enriched_data_all_genres['Genres Reduced'].apply(pd.Series).stack().value_counts()


Drama           38520
Comedy          23758
Romance         12396
Thriller        11546
Action          11241
                ...  
Cars              216
Prison            212
Black Cinema      205
Medical           199
Holidays          171
Length: 78, dtype: int64